[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JorisRoels/deep-learning-biology/blob/main/exercises/assignments/2020-dlb-4-rnn.ipynb)

# Exercise 4: Recurrent Neural Networks

In this notebook, we will be using recurrent neural networks (RNN) to predict secondary structure (SS) from protein sequences. 

The structure of these exercises is as follows: 

1. [Import libraries and download data](#scrollTo=ScagUEMTMjlK)
2. [Data pre-processing](#scrollTo=9ZKz5lBePiZn)
3. [RNN for SS prediction](#scrollTo=TreeLUegygQt)
4. [LSTM for SS prediction](#scrollTo=dYrVpP0s0Zm4)
5. [Extension with convolutional layers](#scrollTo=PHqTVcZrAPhH)

This notebook is largely based on the research published in: 

Torrisi, M., Kaleel, M., & Pollastri, G. (2019). Deeper Profiles and Cascaded Recurrent and Convolutional Neural Networks for state-of-the-art Protein Secondary Structure Prediction. Scientific Reports. https://doi.org/10.1038/s41598-019-48786-x

## 1. Import libraries and download data
Let's start with importing the necessary libraries. 

In [ ]:
import os
import numpy as np
import gdown
import zipfile
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import matplotlib.pyplot as plt
plt.rcdefaults()
from sklearn import metrics
from progressbar import ProgressBar, Percentage, Bar, ETA, FileTransferSpeed

As you will notice, Colab environments come with quite a large library pre-installed. If you need to import a module that is not yet specified, you can add it in the previous cell (make sure to run it again). If the module is not installed, you can install it with `pip`. 

To make your work reproducible, it is advised to initialize all modules with stochastic functionality with a fixed seed. Re-running this script should give the same results as long as the seed is fixed. 

In [ ]:
# make sure the results are reproducible
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# run all computations on the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Running computations with %s' % torch.device(device))
if torch.cuda.is_available():
    print(torch.cuda.get_device_properties(device))

We will now download the required data from a public Google Drive repository. The data is stored as a zip archive and automatically extracted to the `data` directory in the current directory. 

In [ ]:
# fields
url = 'http://data.bits.vib.be/pub/trainingen/DeepLearning/data-4.zip'
cmp_data_path = 'data.zip'

# download the compressed data
gdown.download(url, cmp_data_path, quiet=False)

# extract the data
zip = zipfile.ZipFile(cmp_data_path)
zip.extractall('')

# remove the compressed data
os.remove(cmp_data_path)

## 2. Data visualization and pre-processing

The data used for this exercise session originates from the Protein Data Bank (PDB), a public repository of all freely and publicly known protein structures. Eight classes of secondary structure were annotated: helices (G, H, I), sheets (E, B) and coils (C, S, T). 

Let's start by extracting and loading the data. This notebook can be relatively RAM intensive. By default, you will only use 30% of the available data. If you notice crashes due to RAM overuse, please reduce the `frac` parameter below. You will use less data, and therefore less RAM, so the results will be slightly worse, but your conclusions should not be affected. 

In [ ]:
# function that loads and preprocesses the dataset
def load_dataset(f, frac=0.5):

    file = open(f, 'r') 

    # read in header
    n_proteins = int(file.readline())
    input_per_AA_classess = file.readline()
    data = file.readlines()

    # select a fraction of the data (due to RAM limitations, should be a multiple of 5)
    max_lines = (int(len(data) * frac) // 5) * 5
    data = data[:max_lines]

    # read in the actual data
    pdb_ids = []
    lengths = []
    encoded_inputs = []
    secondary_structures = []
    widgets = ['Loading data: ', Percentage(), ' ', Bar(), ' ', ETA()]
    pbar = ProgressBar(widgets=widgets, maxval=len(data))
    pbar.start()
    for i, line in enumerate(data):
        line = line.replace('\n', '')
        if i % 5 == 0:
            pdb_ids.append(line)
        elif i % 5 == 1:
            length = int(line)
            lengths.append(length)
        elif i % 5 == 2:
            encoded_input = np.asarray(line.split(' '), dtype=float).reshape((length, 22))
            encoded_inputs.append(encoded_input) 
        elif i % 5 == 3:
            secondary_structure = np.asarray(line[:-1].split(' '), dtype=int)
            secondary_structures.append(secondary_structure)
        pbar.update(i)
    pbar.finish()

    # save the lengths as a numpy array
    lengths = np.asarray(lengths, dtype=int)

    # pad all sequences to the length of the longest sequence (with zeros)
    # the corresponding secondary labels are padded with negative values
    max_length = lengths.max()
    widgets = ['Padding data: ', Percentage(), ' ', Bar(), ' ', ETA()]
    pbar = ProgressBar(widgets=widgets, maxval=len(encoded_inputs))
    pbar.start()
    for i, ei in enumerate(encoded_inputs):
        encoded_inputs[i] = np.pad(ei, ((0, max_length - ei.shape[0]), (0, 0)), 'constant', constant_values=0)
        secondary_structures[i] = np.pad(secondary_structures[i], ((0, max_length - secondary_structures[i].shape[0])), 
                                        'constant', constant_values=-1)
        pbar.update(i)
    pbar.finish()

    # save the data in numpy arrays
    encoded_inputs = np.asarray(encoded_inputs)
    secondary_structures = np.asarray(secondary_structures, dtype=int)

    return pdb_ids, encoded_inputs, secondary_structures, lengths

# path to the data
f_train = 'data-4/train-vHHblits.dataset'
f_test = 'data-4/test-vHHblits.dataset'

# load only a fraction due to RAM limitations
frac = 0.3

# load the data
pdb_ids_train, encoded_inputs_train, secondary_structures_train, lengths_train = load_dataset(f_train, frac=frac)
pdb_ids_test, encoded_inputs_test, secondary_structures_test, lengths_test = load_dataset(f_test, frac=frac)

# mappings of the class indices to secondary structure labels
mappings = ['G', 'H', 'I', 'E', 'B', 'C', 'S', 'T']

# print out basic statistics
print()
print('There are %d protein sequences in the training set, with an average length of %.2f. The class distribution is as follows: ' % (encoded_inputs_train.shape[0], lengths_train.mean()))
for i, label in enumerate(mappings):
    print('%s: %.2f%%' % (label, (secondary_structures_train == i).sum() / (secondary_structures_train >= 0).sum() * 100))
print('There are %d protein sequences in the testing set, with an average length of %.2f. The class distribution is as follows: ' % (encoded_inputs_test.shape[0], lengths_test.mean()))
for i, label in enumerate(mappings):
    print('%s: %.2f%%' % (label, (secondary_structures_test == i).sum() / (secondary_structures_test >= 0).sum() * 100))

Even though the data is quite imbalanced, it turns out we can predict one of these eight classes relatively accurate (see the paper reference at the start of this notebook). In this tutorial, however, we will simplify the problem a little bit by merging the secondary structures up to the level of helices, sheets and coils. The following block of code performs this merging step and prints out the class distribution. 

In [ ]:
# group labels to helices, sheets and coils for simplicity
secondary_structures_group_train = np.copy(secondary_structures_train)
helices = (secondary_structures_train == 0) + (secondary_structures_train == 1) + (secondary_structures_train == 2)
secondary_structures_group_train[helices] = 0
sheets = (secondary_structures_train == 3) + (secondary_structures_train == 4)
secondary_structures_group_train[sheets] = 1
coils = (secondary_structures_train == 5) + (secondary_structures_train == 6) + (secondary_structures_train == 7)
secondary_structures_group_train[coils] = 2

# same for test set
secondary_structures_group_test = np.copy(secondary_structures_test)
helices = (secondary_structures_test == 0) + (secondary_structures_test == 1) + (secondary_structures_test == 2)
secondary_structures_group_test[helices] = 0
sheets = (secondary_structures_test == 3) + (secondary_structures_test == 4)
secondary_structures_group_test[sheets] = 1
coils = (secondary_structures_test == 5) + (secondary_structures_test == 6) + (secondary_structures_test == 7)
secondary_structures_group_test[coils] = 2

# print out statistics of the grouped labels
mappings_hl = ['helices', 'sheets', 'coils']
print()
print('The grouped class distribution in the training set is as follows: ')
for i, label in enumerate(mappings_hl):
    print('%s: %.2f%%' % (label, (secondary_structures_group_train == i).sum() / (secondary_structures_group_train >= 0).sum() * 100))
print('The grouped class distribution in the training set is as follows: ')
for i, label in enumerate(mappings_hl):
    print('%s: %.2f%%' % (label, (secondary_structures_group_test == i).sum() / (secondary_structures_group_test >= 0).sum() * 100))

It seems that the sheets class is slightly under-represented. We will correct this by using the following class weights: 
$$w_c = \frac{N}{C \times N_c}$$
where $N=4725$ is the total amount of training samples, $C=3$ the amount of classes and $N_c$ is the amount of samples that belong to class $c$. 

In [ ]:
# computing balancing weights
t = (secondary_structures_group_train >= 0).sum()
w = [t / (3 * (secondary_structures_group_train == i).sum()) for i in range(3)]
w = np.asarray(w)
print()
print('Class balancing weights: ')
for i, label in enumerate(mappings_hl):
    print('%s: %.2f' % (label, w[i]))

As you can see, the sheets class will be assigned a larger weight to cope with its slight under-representation. 

Now that we have our data in order, it is time to define a dataloader that can be used for our predictive recurrent networks. The dataloader should return sequences of encodings and their corresponding labels. 

In [ ]:
batch_size = 256

train = TensorDataset(torch.from_numpy(encoded_inputs_train).float(), 
                      torch.from_numpy(secondary_structures_group_train))
test = TensorDataset(torch.from_numpy(encoded_inputs_test).float(), 
                     torch.from_numpy(secondary_structures_group_test))

dataloader_train = DataLoader(train, batch_size=batch_size)
dataloader_test = DataLoader(test, batch_size=batch_size)

In [ ]:
i = 2
x, y = train[i]
x = x[:lengths_train[i]]
y = y[:lengths_train[i]]
print('Sequence %d:' % i)
print('    - input shape = %s' % str(list(x.shape)))
print('    - output shape = %s' % str(list(y.shape)))
print('    - length = %d' % x.shape[0])
print('    - AA-encoding length = %d' % x.shape[1])
print('    - padded length = %d' % train[i][0].shape[0])
for i in range(3):
    plt.scatter(np.arange(len(y))[y==i], y[y==i])
plt.legend(mappings_hl)
plt.yticks(np.arange(3))
plt.show()

This plot shows the dimensions of a particular input sequence. Each amino-acid (AA) is encoded as a vector of length $A=22$. Each AA has a corresponding secondary structure (helix, sheet or coil). The protein sequences are variable length, however, we have padded the sequences to match with the longest sequence. The corresponding secondary structure labels for these paddings were set to -1. 

## 3. RNN for SS prediction

Now we are ready to define a recurrent network that can process the protein sequences and predict the corresponding SS label. The model will consist of an RNN that takes an input sequence and maps this to a hidden state. This hidden state can then be fed to a linear layer to predict the SS of the protein sequence. You can chose the dimensionality of the hidden state yourself. 

**Exercise**: implement the `RNNNet` class, more specifically: 
- Implement the initializer by defining the correct layers: you will need an [RNN](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html) layer and a [Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) layer that performs classification. 
- Implement the forward propagation function. 
- Evaluate the performance by plotting the learning curves and showing the confusion matrix

In [ ]:
class RNNNet(nn.Module):
    
    def __init__(self):
        super(RNNNet, self).__init__()

        """
        INSERT CODE HERE
        """

    def forward(self, x):
        
        """
        INSERT CODE HERE
        """
        
        return x

rnn = RNNNet().to(device)
print(rnn)

You can use the code below to train and evaluate your implemented model. For a given input sequence, your model should return a sequence of class probability distributions. More precisely, for a batch size $B$ and sequence of length $S$, the input has shape $[B, S, A]$ and the output has shape $[B, S, C]$. 

In [ ]:
# evaluates the performance of a model and return the predictions and ground truth
def evaluate(model, loader):
    model.eval()
    outputs = []
    predictions = []
    for i, batch in enumerate(loader):
        sequences = batch[0].to(device)
        output = batch[1].to(device)
        prediction = model(sequences)
        
        output = output.flatten()
        mask = output >= 0
        output = output[mask]
        prediction = prediction.argmax(dim=2).flatten()
        prediction = prediction[mask]
        
        predictions.append(prediction.detach().cpu().numpy())
        outputs.append(output.detach().cpu().numpy())
    
    return np.hstack(predictions), np.hstack(outputs)

# implementation of a single training epoch
def train_epoch(net, loader, loss_fn, optimizer):
    
    # set the network in training mode
    net.train()
    
    # keep track of the loss
    loss_cum = 0
    cnt = 0
    
    for i, data in enumerate(loader): 
        
        # sample data
        x, y = data
        x = x.to(device)
        y = y.to(device)
        
        # set all gradients equal to zero
        net.zero_grad()
        
        # feed the batch to the network and compute the outputs
        y_pred = net(x)
        
        # compare the outputs to the labels with the loss function
        loss = loss_fn(y_pred.view(-1, y_pred.size(2)), y.flatten())
        loss_cum += loss.data.cpu().numpy()
        cnt += 1
        
        # backpropagate the gradients w.r.t. computed loss
        loss.backward()

        # apply one step in the optimization
        optimizer.step()
    
    # compute the average loss
    loss_avg = loss_cum / cnt
    
    return loss_avg

# implementation of a single testing epoch
def test_epoch(net, loader, loss_fn):
    
    # set the network in evaluation mode
    net.eval()
    
    # keep track of the loss
    loss_cum = 0
    cnt = 0
    
    for i, data in enumerate(loader): 
        
        # sample data
        x, y = data
        x = x.to(device)
        y = y.to(device)
        
        # feed the batch to the network and compute the outputs
        y_pred = net(x)
        
        # compare the outputs to the labels with the loss function
        loss = loss_fn(y_pred.view(-1, y_pred.size(2)), y.flatten())
        loss_cum += loss.data.cpu().numpy()
        cnt += 1
    
    # compute the average loss
    loss_avg = loss_cum / cnt
    
    return loss_avg

def train_net(net, train_loader, test_loader, loss_fn, optimizer, epochs):
    
    # transfer the network to the GPU
    net = net.to(device)
    
    train_loss = np.zeros((epochs))
    test_loss = np.zeros((epochs))
    train_acc = np.zeros((epochs))
    test_acc = np.zeros((epochs))
    for epoch in range(epochs):
        
        # training
        train_loss[epoch] = train_epoch(net, train_loader, loss_fn, optimizer)
        predictions, outputs = evaluate(net, train_loader)
        train_acc[epoch] = metrics.balanced_accuracy_score(outputs, predictions)
        
        # testing
        test_loss[epoch] = test_epoch(net, test_loader, loss_fn)
        predictions, outputs = evaluate(net, test_loader)
        test_acc[epoch] = metrics.balanced_accuracy_score(outputs, predictions)
        
        print('Epoch %5d - Train loss: %.6f - Train accuracy: %.6f - Test loss: %.6f - Test accuracy: %.6f' 
              % (epoch, train_loss[epoch], train_acc[epoch], test_loss[epoch], test_acc[epoch]))
    
    return (train_loss, test_loss), (train_acc, test_acc)

epochs = 10
lr = 1e-2

loss_fn = nn.CrossEntropyLoss(ignore_index=-1, weight=torch.from_numpy(w).to(device).float())
optimizer = optim.Adam(rnn.parameters(), lr=lr)

loss_rnn, acc_rnn = train_net(rnn, dataloader_train, dataloader_test, loss_fn, optimizer, epochs)

In [ ]:
plt.rcParams["figure.figsize"] = (15,5)

# plot train/test loss curves
"""
INSERT CODE HERE
"""

# plot train/test balanced accuracy
"""
INSERT CODE HERE
"""

# show the confusion matrix
"""
INSERT CODE HERE
"""

That seems not too bad. You should be able to achieve a test balanced accuracy of about 60%. The training curve also indicates that we might be able to get higher test accuracy by training it a little longer. 

## 3. LSTM for SS prediction

Now that we have a working RNN, we can try and improve it by changing it into an LSTM. 

**Exercise**: implement the `LSTMNet` class, more specifically: 
- Implement the initializer by defining the correct layers: you will need an [LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) layer and a [Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) layer that performs classification. 
- Implement the forward propagation function. 
- Make the LSTM bidirectional and evaluate the performance
- Change the hidden size and evaluate the performance
- Change the number of layers in the LSTM, i.e. the amount of LSTMs stacked on top of each other and evaluate the performance. 

In [ ]:
class LSTMNet(nn.Module):
    
    def __init__(self, hidden_size=16, bidirectional=False, num_layers=1):
        super(LSTMNet, self).__init__()

        """
        INSERT CODE HERE
        """

    def forward(self, x):

        """
        INSERT CODE HERE
        """
        
        return x


"""
INSERT CODE HERE
"""

In [ ]:
plt.rcParams["figure.figsize"] = (15,5)

# plot test loss curves
"""
INSERT CODE HERE
"""

# plot test balanced accuracy
"""
INSERT CODE HERE
"""

# show the confusion matrix of the best performing model
"""
INSERT CODE HERE
"""

It could take a while to train all these different networks, so preferably chose a sufficiently high learning rate. From the experiments, it turns out that the LSTM network is outperforming the standard RNN network. Given the theory, this is also according to our expectations. Moreover, we see that wider hidden units and stacked LSTMs can benefit the results. However, the bidirectional LSTMs turn out to improve the results the most in this case. The best performing method that combines all these improvements (wider, stacked and bidirectional) scores a balanced accuracy of over 70%. 

## 4. Extension with convolutional layers

Convolutional layers can be used to improve the model that we currently have. The idea is that a convolutional layer can provide a good embedding and that the recurrent network exploits this. The architecture that we will implement works as follows: the input is fed to a ReLU activated 1D convolutional layer (along the sequence dimension), a recurrent network (we will use an LSTM) and two linear layers for classification (the first one is ReLU activated). The figure below shows the architecture. 

![Image](https://i.ibb.co/MgVtHgH/scheme.png)

**Exercise**: Implement the architecture described in the top figure. You will need a [Conv1d](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html), [ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html), [LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) layer and two [Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) layers. Make sure the convolution is applied along the sequence dimension. The [`permute`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.permute) function can be useful for this. Evaluate the performance and compare to the previous models. 

In [ ]:
class ImprovedLSTMNet(nn.Module):
    
    def __init__(self, hidden_size=16, bidirectional=False, num_layers=1, conv_channels=16, linear_channels=16):
        super(ImprovedLSTMNet, self).__init__()

        """
        INSERT CODE HERE
        """

    def forward(self, x):
        
        """
        INSERT CODE HERE
        """
        
        return x

lstm = ImprovedLSTMNet(bidirectional=True, hidden_size=32, num_layers=2).to(device)

print(lstm)
optimizer = optim.Adam(lstm.parameters(), lr=lr)

loss_lstm, acc_lstm = train_net(lstm, dataloader_train, dataloader_test, loss_fn, optimizer, epochs)

In [ ]:
plt.rcParams["figure.figsize"] = (15,5)

# plot test loss curves
"""
INSERT CODE HERE
"""

# plot test balanced accuracy
"""
INSERT CODE HERE
"""

# show the confusion matrix of the best performing model
"""
INSERT CODE HERE
"""

Looks like we have improved the results to 75%! Convolutional layers seem to be great embedding layers for sequence data. It turns out that you can actually achieve 84% accuracy on this dataset (Torrisi et.al., 2019), by using all available training data (we only used a fraction due to RAM limits), increasing the depth/width of the model, more epochs, etc. Feel free to try this out on your own if you have the resources available! 

Note that your results might differ a little due to the different initialization that you might have used. By fixing the seed in the first code cell and running everything from scratch, the results should be reproducible. 